In [ ]:
"""
Example usage of the EODHD API client.

This notebook demonstrates how to use the EODHDClient to fetch financial news articles
from the EODHD API with different parameters and display the results.
"""

import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
import pandas as pd
from financial_news_rag.eodhd import EODHDClient

# Load environment variables
load_dotenv()

# Create an EODHD client
client = EODHDClient()

# Define date range for the last 7 days
today = datetime.now()
week_ago = today - timedelta(days=7)
today_str = today.strftime("%Y-%m-%d")
week_ago_str = week_ago.strftime("%Y-%m-%d")

# Define date range for the last 30 days
month_ago = today - timedelta(days=30)
month_ago_str = month_ago.strftime("%Y-%m-%d")

# Define date range for the last year
year_ago = today - timedelta(days=365)
year_ago_str = year_ago.strftime("%Y-%m-%d")


In [ ]:

# Example 1: Fetch news by tag (Mergers and Acquisitions)
print("Fetching news articles about Mergers and Acquisitions...")
merger_news = client.fetch_news(
    tag="MERGERS AND ACQUISITIONS",
    from_date=week_ago_str,
    to_date=today_str,
    limit=10
)


In [ ]:

# Display results
print(f"\nFound {len(merger_news)} articles about Mergers and Acquisitions\n")
for i, article in enumerate(merger_news):
    print(f"Article {i+1}:")
    print(f"Title: {article['title']}")
    print(f"Published: {article['published_at']}")
    print(f"URL: {article['url']}")
    print(f"Symbols: {', '.join(article['symbols']) if article['symbols'] else 'None'}")
    print(f"Tags: {', '.join(article['tags'])}")
    print(f"Sentiment: {article['sentiment']}")
    print(f"Content: {article['content']}\n")


In [ ]:
merger_news

In [ ]:
# Example 2: Fetch news for a specific symbol (Apple)
print("\nFetching news for Apple...")
try:
    # Note: EODHD API only supports a single symbol at a time
    tech_news = client.fetch_news(
        symbols="AAPL.US",  # Must be a single symbol
        from_date=year_ago_str,
        to_date=today_str,
        limit=10
    )
    
    # Display results
    print(f"\nFound {len(tech_news)} articles about Apple\n")
    for i, article in enumerate(tech_news):
        print(f"Article {i+1}:")
        print(f"Title: {article['title']}")
        print(f"Published: {article['published_at']}")
        print(f"URL: {article['url']}")
        print(f"Symbols: {', '.join(article['symbols']) if article['symbols'] else 'None'}")
        print(f"Tags: {', '.join(article['tags']) if article['tags'] else 'None'}")
        print(f"Sentiment: {article['sentiment']}")
        print(f"Summary: {article['content'][:150]}...\n")
except Exception as e:
    print(f"Error fetching tech news: {e}")


In [ ]:
tech_news

In [ ]:
# Example 3: Fetching news for multiple symbols (requires separate API calls)
print("\nFetching news for multiple symbols (Microsoft, Google)...")
try:
    # For multiple symbols, we need to make separate API calls for each
    symbols_to_query = ["MSFT.US", "GOOG.US"]
    multi_symbol_news = []
    
    for symbol in symbols_to_query:
        print(f"Fetching news for {symbol}...")
        articles = client.fetch_news(
            symbols=symbol,
            from_date=month_ago_str,
            to_date=today_str,
            limit=5
        )
        print(f"Found {len(articles)} articles for {symbol}")
        multi_symbol_news.extend(articles)
    
    print(f"\nTotal articles found across all symbols: {len(multi_symbol_news)}")
    
    # Display a summary of the results
    symbol_counts = {}
    for article in multi_symbol_news:
        if article['symbols']:
            for symbol in article['symbols']:
                symbol_counts[symbol] = symbol_counts.get(symbol, 0) + 1
        else:
            symbol_counts['None'] = symbol_counts.get('None', 0) + 1
    
    print("\nArticles by symbol mentioned:")
    for symbol, count in sorted(symbol_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"- {symbol}: {count}")
except Exception as e:
    print(f"Error fetching multi-symbol news: {e}")

# Example 4: Convert results to DataFrame for analysis
print("\nConverting results to DataFrame for analysis...")
try:
    # Combine all result sets - ensure we have data to work with
    all_articles = []
    if 'merger_news' in locals() and merger_news:
        all_articles.extend(merger_news)
    if 'tech_news' in locals() and tech_news:
        all_articles.extend(tech_news)
    if 'multi_symbol_news' in locals() and multi_symbol_news:
        all_articles.extend(multi_symbol_news)
        
    if all_articles:
        df = pd.DataFrame(all_articles)
        
        # Show DataFrame information
        print(f"\nDataFrame Shape: {df.shape}")
        print("\nColumns:")
        for col in df.columns:
            print(f"- {col}")
        
        # Basic analysis
        if not df.empty:
            print("\nArticles per tag:")
            tag_counts = {}
            for article in df.to_dict('records'):
                if article['tags']:  # Check if tags list is not empty
                    for tag in article['tags']:
                        tag_counts[tag] = tag_counts.get(tag, 0) + 1
                else:
                    tag_counts['None'] = tag_counts.get('None', 0) + 1
            
            for tag, count in sorted(tag_counts.items(), key=lambda x: x[1], reverse=True):
                print(f"- {tag}: {count}")
            
            # Sentiment analysis
            print("\nAverage sentiment polarity:", df['sentiment'].apply(lambda x: x.get('polarity', 0)).mean())
    else:
        print("No articles available for analysis")
except Exception as e:
    print(f"Error creating DataFrame: {e}")
